In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from webdriver_manager.firefox import GeckoDriverManager

from selenium.webdriver.edge.service import Service as EdgeService
from webdriver_manager.microsoft import EdgeChromiumDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time
import logging
import os

In [2]:
import pandas as pd
from datetime import datetime

autentica = pd.read_excel('ls.xlsx')
login = str(autentica['login'][0])
senha = str(autentica['senha'][0])

name = 'cjbg_suap'
indisponivel = 'indisponivel'

data_e_hora_atuais = datetime.now()
data_e_hora = data_e_hora_atuais.strftime('%Y-%m-%d_%H-%M')

log_dir = 'logs'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [3]:
logger = logging.getLogger('extração')
logger.setLevel(logging.DEBUG)

fHandler = logging.FileHandler(f'logs/{data_e_hora}_extracao.log', mode='w')
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

fHandler.setFormatter(formatter)
logger.addHandler(fHandler)

# logging.basicConfig(level=logging.DEBUG, filename='app.log', filemode='x', format='%(asctime)s - %(levelname)s - %(message)s')
logger.info(f'iniciando extarção campus {name}')
# logger.debug('debug message')
# logger.warning('warn message')

In [4]:

def abrir_suap():
    logger.info('Iniciando navegador')
    global nav
    # nav = webdriver.Firefox(executable_path=GeckoDriverManager().install())
    # nav = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    nav = webdriver.Edge(service=EdgeService(EdgeChromiumDriverManager().install()))
    # nav = webdriver.Chrome(ChromeDriverManager(version='114.0.5735.90').install())
    nav.set_page_load_timeout(50)
    logger.info('acessando SUAP')
    nav.get("https://suap.ifpe.edu.br/accounts/login/?next=/")
    
    # inserindo usuário
    logger.info('inserindo nome e senha')
    nav.find_element(By.XPATH, '//*[@id="id_username"]').click()
    nav.find_element(By.XPATH, '//*[@id="id_username"]').send_keys(login)

    # inserindo senha 
    nav.find_element(By.XPATH, '//*[@id="id_password"]').click()
    nav.find_element(By.XPATH, '//*[@id="id_password"]').send_keys(senha)

    nav.find_element(By.XPATH, '//*[@id="ok"]').click()

In [5]:
# extraindo para lista
def table_to_list():
    logger.info('capturando tabela e convertendo em lista')
    tabela = []

    lista = nav.find_element(By.TAG_NAME, 'table')
    td = lista.find_elements(By.TAG_NAME, 'td')

    for i in td:
        tabela.append(i.text)

    return tabela

In [6]:
def build_df(lista):
    
    logger.info('montando DatFrame')
    df_tabela = pd.DataFrame()


    tombos = []
    ele_des = []
    descricao = []
    cargas = []
    c_contabil = []
    salas = []
    dt_entrada = []
    fornecedores = []
    valores = []

    #  remove o primeiro elemento ta tabela, informação quebrada    
    for excluir in range(0, len(lista), 1201):
        lista.pop(excluir)

    for tombo in range(1, len(lista)+1, 12):
        if lista[tombo] == '':
            lista.pop(tombo)

    # listas para montar o data frame
    for tombo in range(1, len(lista), 12):
        tombos.append(lista[tombo])

    for ed in range(2, len(lista), 12):
        ele_des.append(lista[ed])

    for des in range(3, len(lista), 12):
        descricao.append(lista[des])

    for carga in range(4, len(lista), 12):
        cargas.append(lista[carga])

    for c in range(5, len(lista), 12):
        c_contabil.append(lista[c])

    for sala in range(6, len(lista), 12):
        salas.append(lista[sala])

    for data in range(8, len(lista), 12):
        dt_entrada.append(lista[data])
    
    for f in range(10, len(lista), 12):
        fornecedores.append(lista[f])

    for valor in range(11, len(lista), 12):
        valores.append(lista[valor])



    #  montagem do data frame
    df_tabela['tombo'] = tombos
    df_tabela['tombo'] = df_tabela['tombo'].astype(str).str[:-6]

    df_tabela['ed'] = ele_des

    df_tabela['descricao'] = descricao

    df_tabela['carga'] = cargas
    df_tabela['sala'] = salas

    df_tabela['c_contabil'] = c_contabil

    df_tabela['dt_entrada'] = dt_entrada
#     df_tabela['dt_entrada'] = pd.to_datetime(df_tabela['dt_entrada'], infer_datetime_format=True)

    df_tabela['fornecedor'] = fornecedores
    
    df_tabela['valor'] = valores
    # df_tabela['valor'] = df_tabela['valor'].str.replace('.','',regex=True)
    # df_tabela['valor'] = df_tabela['valor'].str.replace(',','.',regex=True)
    # df_tabela['valor'] = pd.to_numeric(df_tabela['valor'])
    # df_tabela['valor'] = df_tabela['valor'].round(2)
    logger.info('DataFrame montado')
    return df_tabela

In [7]:
def pagination():
    
    abrir_suap()
    nav.set_page_load_timeout(60)
    print(f'Extração iniciada as: {datetime.now()}')
    pag = 1
    campi = 14
    data_final = '16%2F10%2F2024'
    saida = False
    complete_df = pd.DataFrame()
    logger.info('preparando url search params ')
    
    while saida is False:
    # while pag < 3:
    # cafi->12  crec->4x    cbja->7x   cbar->8x   ccar->11x    ccsa->13x    cgar->10x   cabl->22x 
    # cipj->6   cjbg->14   coli->20x  cpes->5x   cplt->21x    cpmr->18    cvsa->9x    reipe->3x
        try:
            logger.info(f'paginando pagina-{pag} ')
            nav.get(f'https://suap.ifpe.edu.br/patrimonio/inventario_busca/?page={pag}&status=&empenho=&empenho=&data_final={data_final}&sala=&numero_serie=&numero=&rotulos=&data_inicial=&estado_conservacao=6&elemento_despesa=&valor=&responsavel=&responsavel=&contabil=&fornecedor=&fornecedor=&setor_responsavel=&campus={campi}&descricao=')
            celula = nav.find_element(By.CLASS_NAME, 'row1')

            if celula:
                suap_list = table_to_list()
                suap_df = build_df(suap_list)
                complete_df = pd.concat([complete_df, suap_df], ignore_index= True)
                pag += 1

            else:
                saida = True

        except NoSuchElementException:
            
            try:
                error = nav.find_element(By.ID, 'main-frame-error')
                print(f'Caiu as: {datetime.now()} - na pagina {pag}')
                logger.info(f'Caiu as: {datetime.now()} - na pagina {pag}')
                
                if error:

                    nav.quit()
                    abrir_suap()
                    print(f'Retomou as: {datetime.now()} na pagina {pag}')
                    logger.info(f'Retomou as: {datetime.now()} na pagina {pag}')
                    nav.get(f'https://suap.ifpe.edu.br/patrimonio/inventario_busca/?page={pag}&status=&empenho=&empenho=&data_final=&sala=&numero_serie=&numero=&rotulos=&data_inicial=&estado_conservacao=6&elemento_despesa=&valor=&responsavel=&responsavel=&contabil=&fornecedor=&fornecedor=&setor_responsavel=&campus={campi}&descricao=')
                    celula = nav.find_element(By.CLASS_NAME, 'row1')

                    if celula:
                        pag += 1
                        suap_list = table_to_list()
                        suap_df = build_df(suap_list)
                        complete_df = pd.concat([complete_df, suap_df], ignore_index= True)
                        print(f'pagina pos erro - {pag}')
                        logger.info(f'pagina pos erro - {pag}')
                        continue

                    else:
                        break
                        
            except NoSuchElementException:
                print('acabaram as paginas')
                logger.info('acabaram as paginas')
                break
            
                

    print(f'Extração finalizada as: {datetime.now()} na pagina {pag}')
    logger.info(f'Extracao finalizada as: {datetime.now()} na pagina {pag}')
    # complete_df["valor"] = complete_df["valor"].str.replace('.','',regex=True)
    # complete_df["valor"] = complete_df["valor"].str.replace(',','.',regex=True)
    # complete_df["valor"] =  pd.to_numeric(complete_df["valor"])
    # complete_df["valor"] = complete_df["valor"].round(2)
    
    print(complete_df.info())
    display(complete_df)
    logger.info(f'exportando arquivo .xlsx')
    complete_df.to_excel(f'extracoes_suap/{data_e_hora}_{name}_relatorio_patrimonio.xlsx', index=False)
    return complete_df
    
#     nav.quit()
    

In [8]:
def complement_item(item_list):
    
    abrir_suap()
    tombo = []
    status = []
    ed = []
    conservation = []
    carga = []
    desc = []
    empenho = []
    ug_ne = []
    uasg = []
    nf = []
    dt_nf = []
    type_entry = []
    supplier = []
    process = []
    linha = 0
    c = 1
    
    
    print(f'Iniciou as: {datetime.now()}')
    
#     for linha in item_list:
    while linha < (len(item_list)):
        
        print(f'index - {linha} -> tombo - {item_list[linha]}')
        
        try:
        
            
            nav.get(f"https://suap.ifpe.edu.br/patrimonio/inventario/{item_list[linha]}/")
            
        
            tombo.append(item_list[linha])

            st = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[2]/td[2]").get_attribute('innerText')
            status.append(st)
            print(f'st - {st}')

            ele_des = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[5]/td[2]").get_attribute('innerText')
            ed.append(ele_des)
            print(f'ele_des - {ele_des}')

            con = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[4]/td[2]").get_attribute('innerText')
            conservation.append(con)
            print(f'con - {con}')

            cargo = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[7]/td[2]").get_attribute('innerText')
            carga.append(cargo)
            print(f'cargo - {cargo}')

            descricao = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[6]/td[2]/span[1]").get_attribute('innerText')
            desc.append(descricao)
            print(f'descricao - {descricao}')

            try:
                nav.set_page_load_timeout(40)
                nav.find_element(By.LINK_TEXT, "Detalhes da Entrada").click()

            
                ne = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[12]/td/a").get_attribute('innerText')
                ug = ne[18:22]
                ne = ne[:12]
                empenho.append(ne)
                ug_ne.append(ug)
                print(f'ne - {ne}')
                print(f'ug - {ug}')
            # print(empenho)
            except NoSuchElementException as ex:
                empenho.append(indisponivel)
                ug_ne.append(indisponivel)
                print(f'entrou no except do empenho')

            try:
                gest = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[1]/td").get_attribute('innerText')
                uasg.append(gest)
                print(f'gest - {gest}')
            except NoSuchElementException as ex:
                uasg.append(indisponivel)
                print(f'entrou no print da gestao')
                # print(uasg)

            try:
                note = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[4]/td").get_attribute('innerText')
                nf.append(note)
                print(f'ne - {note}')
            except NoSuchElementException as ex:
                nf.append(indisponivel)
                print(f'entrou no expect da NF')
            # print(nf)

            try:
                data = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[5]/td").get_attribute('innerText')
                dt_nf.append(data)
                print(f'data - {data}')
            except NoSuchElementException as ex:
                dt_nf.append(indisponivel)
                print(f'entrou no except da data')
            # print(data)

            try:
                entry = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[3]/td").get_attribute('innerText')
                type_entry.append(entry)
                print(f'entrada - {entry}')
            except NoSuchElementException as ex:
                type_entry.append(indisponivel)
                print(f'entrou no print da entrada')

            try:
                sp = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[6]/td/a").get_attribute('innerText')
                supplier.append(sp)
                print(f'fornecedor - {sp}')
            except NoSuchElementException as ex:
                supplier.append(indisponivel)
                print(f'entrou no print do fornecedor')

            try:
                pc = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[13]/td").get_attribute('innerText')
                process.append(pc)
                print(f'process - {pc}\n\n')
            except NoSuchElementException as ex:
                process.append(indisponivel)
                print(f'entrou bo print do processo\n\n')
                
            linha += 1
            continue

        except TimeoutException:
            # print(indisponivel)
            
            print(f'entrou no TimeoutException {c} vezes\n\n')
            empenho.append(indisponivel)
            ug_ne.append(indisponivel)
            uasg.append(indisponivel)
            nf.append(indisponivel)
            dt_nf.append(indisponivel)
            type_entry.append(indisponivel)
            supplier.append(indisponivel)
            process.append(indisponivel)
            print(f'TimeoutException as: {datetime.now()} no item {item_list[linha]}\n\n')

            linha += 1
            c += 1
            continue
        
        except NoSuchElementException as ex:
            
            try:
                    error = nav.find_element(By.ID, 'summary')
                    print(f'erro de tombo as: {datetime.now()} - no item {item_list[linha]}')
                    
                    if error:
                        status.append(indisponivel)
                        ed.append(indisponivel)
                        conservation.append(indisponivel)
                        carga.append(indisponivel)
                        desc.append(indisponivel)
                        empenho.append(indisponivel)
                        ug_ne.append(indisponivel)
                        uasg.append(indisponivel)
                        nf.append(indisponivel)
                        dt_nf.append(indisponivel)
                        type_entry.append(indisponivel)
                        supplier.append(indisponivel)
                        process.append(indisponivel)
                        print(f'ValueError as: {datetime.now()} no item {item_list[linha]}')
                        linha += 1
                        continue
            
            except NoSuchElementException as ex:
                
                try:
                    error = nav.find_element(By.ID, 'id_username')
                    print(f'Caiu as: {datetime.now()} - no item {item_list[linha]}')

                    if error:

                        nav.quit()
                        abrir_suap()
                        nav.get(f"https://suap.ifpe.edu.br/patrimonio/inventario/{item_list[linha]}/")
                        print(f'Retomou as: {datetime.now()} no item {item_list[linha]}')
                        continue

                except Exception as ex:
                    linha += 1
                    print('acabaram as paginas')
                    continue

    
            
    table = pd.DataFrame()
    table['tombo'] = tombo
    table['status'] = status
    table['consetvation'] = conservation
    table['carga'] = carga
    table['desc'] = desc
    table['ed'] = ed
    
    table['empenho'] = empenho
    table['ug_empenho'] = ug_ne
    table['uasg'] = uasg
    table['nt_fiscal'] = nf
    table['dt_nf'] = dt_nf
    table['tipo_entrada'] = type_entry
    table['fornecedor'] = supplier
    table['processo'] = process
    
    for i in range(0, len(table['tipo_entrada'])):
        if 'Doacao' in table['tipo_entrada'][i]:
            table['dt_nf'][i] = indisponivel
            table['nt_fiscal'][i] = indisponivel
            
    print(f'tombo {len(tombo)}')            
    print(f'status {len(status)}')
    print(f'conser {len(conservation)}')
    print(f'carga {len(carga)}')
    print(f'descricao {len(desc)}')
    print(f'ed {len(ed)}')
    print(f'NE {len(empenho)}')
    print(f'ug {len(ug_ne)}')
    print(f'uasg {len(uasg)}')
    print(f'NF {len(nf)}')
    print(f'dtNF {len(dt_nf)}')
    print(f'tipo ent {len(type_entry)}')
    print(f'fornece {len(supplier)}')
    print(f'processo {len(process)}')
#     print(f'status {status}')
#     print(f'conser {conservation}')
#     print(f'carga {carga}')
#     print(f'descricao {desc}')
#     print(f'ed {ed}')
#     print(f'NE {empenho}')
#     print(f'ug {ug_ne}')
#     print(f'uasg {uasg}')
#     print(f'NF {nf}')
#     print(f'dtNF {dt_nf}')
#     print(f'tipo ent {type_entry}')
#     print(f'fornece {supplier}')
#     print(f'processo {process}')
    print(f'Terminou as: {datetime.now()}')
    print(table.info())
    
    table.to_excel(f'{data_e_hora}_{name}.xlsx', index=False)
        

#     nav.quit()

In [9]:
suap = pagination()

Extração iniciada as: 2024-10-18 13:16:36.171911
acabaram as paginas
Extração finalizada as: 2024-10-18 13:51:59.179160 na pagina 59
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5717 entries, 0 to 5716
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tombo       5717 non-null   object
 1   ed          5717 non-null   object
 2   descricao   5717 non-null   object
 3   carga       5717 non-null   object
 4   sala        5717 non-null   object
 5   c_contabil  5717 non-null   object
 6   dt_entrada  5717 non-null   object
 7   fornecedor  5717 non-null   object
 8   valor       5717 non-null   object
dtypes: object(9)
memory usage: 402.1+ KB
None


,tombo,ed,descricao,carga,sala,c_contabil,dt_entrada,fornecedor,valor
0,82053,449052-12,APARELHO DE AR CONDICIONADO MODELO SPLIT 18.00...,FRANCISCO DO NASCIMENTO JÚNIOR (CJBG JBG-DG),CJBG - ALMOXARIFADO SALA 2 - ALMOXARIFADO - BL...,JABOATAO DOS GUARARAPES,10/06/2011,AVANT COMERCIO LTDA. (12.561.223/0001-07),"1.465,00"
1,99259,449052-35,MICROCOMPUTADOR DESKTOP DIMENSÕES REDUZIDAS (A...,Antônio José da Silva Júnior (CJBG JBG-CTIC),CJBG - SALA CTIC - BLOCO ADMINISTRATIVO TERREO...,JABOATAO DOS GUARARAPES,07/12/2011,ZMAX INDUSTRIA E COMERCIO LTDA (05.137.364/000...,"456,00"
2,99261,449052-35,MICROCOMPUTADOR DESKTOP DIMENSÕES REDUZIDAS (A...,FRANCISCO DO NASCIMENTO JÚNIOR (CJBG JBG-DG),CJBG - SALA CTIC - BLOCO ADMINISTRATIVO TERREO...,JABOATAO DOS GUARARAPES,07/12/2011,ZMAX INDUSTRIA E COMERCIO LTDA (05.137.364/000...,"456,00"
3,99262,449052-35,MICROCOMPUTADOR DESKTOP DIMENSÕES REDUZIDAS (A...,ISAQUE FERREIRA JANUÁRIO (CJBG JBG-CTIC),CJBG - SALA CTIC - BLOCO ADMINISTRATIVO TERREO...,JABOATAO DOS GUARARAPES,07/12/2011,ZMAX INDUSTRIA E COMERCIO LTDA (05.137.364/000...,"456,00"
4,99263,449052-35,MICROCOMPUTADOR DESKTOP DIMENSÕES REDUZIDAS (A...,FRANCISCO DO NASCIMENTO JÚNIOR (CJBG JBG-DG),CORREDOR - BLOCO ADMINISTRATIVO TERREO (CJBG),JABOATAO DOS GUARARAPES,07/12/2011,ZMAX INDUSTRIA E COMERCIO LTDA (05.137.364/000...,"456,00"
...,...,...,...,...,...,...,...,...,...
5712,281494,449052-18,TEXTOS BASICOS DE FILOSOFIA DOS PRE-S0CRATICOS...,THIAGO LEITE AMARO DA SILVA (CJBG JBG-CBIM),,JABOATAO DOS GUARARAPES,18/06/2024,INSTITUTO NACIONAL VERITAS DE CULTURA LTDA (07...,"46,80"
5713,281495,449052-18,TEXTOS BASICOS DE FILOSOFIA DOS PRE-S0CRATICOS...,THIAGO LEITE AMARO DA SILVA (CJBG JBG-CBIM),,JABOATAO DOS GUARARAPES,18/06/2024,INSTITUTO NACIONAL VERITAS DE CULTURA LTDA (07...,"46,80"
5714,281676,449052-18,REACT NATIVE: DESENVOLVIMENTO DE APLICATIVOS M...,THIAGO LEITE AMARO DA SILVA (CJBG JBG-CBIM),,JABOATAO DOS GUARARAPES,18/07/2024,INSTITUTO NACIONAL VERITAS DE CULTURA LTDA (07...,"52,49"
5715,281677,449052-18,REACT NATIVE: DESENVOLVIMENTO DE APLICATIVOS M...,THIAGO LEITE AMARO DA SILVA (CJBG JBG-CBIM),,JABOATAO DOS GUARARAPES,18/07/2024,INSTITUTO NACIONAL VERITAS DE CULTURA LTDA (07...,"52,49"
